In [1]:
%pylab qt4
%load_ext Cython

import importlib
import numpy as np
import gzip
import pickle
import os
from glob import glob
import json
from time import time
from skimage import io, color
from scipy.spatial import Delaunay
from matplotlib.widgets import Cursor, Button

from helpers import * 
for x in [wh, dh, fh, gh, ch, mh]:
    reload(x) 

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_cut_2/'
TH_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_cut_and_thresholded_and_clustered/'
TO_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_mesh_2/'

ALPHA = 25


def do_triangulation(bone_pixels):
    indices_of_bone_pixels = np.nonzero(bone_pixels)
    indices_of_bone_pixels = np.vstack(indices_of_bone_pixels).transpose()
    indices_of_bone_pixels[:, 0] = bone_pixels.shape[0] - indices_of_bone_pixels[:,0]
    
    #tb = time()
    triangulation = Delaunay(indices_of_bone_pixels)
    #print(time() - tb)
    to_delete = alpha_shape_unique(triangulation.points[triangulation.simplices], alpha=ALPHA)
    triangulation.simplices = np.delete(triangulation.simplices, to_delete, axis=0)
    #print(time() - tb)
    
    return triangulation

Populating the interactive namespace from numpy and matplotlib
Current base path: /home/stefan/Dropbox/Masterarbeit


In [2]:
%%cython

cimport numpy as np
import numpy as np
cimport cython
from libc.math cimport sqrt


@cython.boundscheck(False)
@cython.wraparound(False)

def alpha_shape_unique(np.ndarray[np.float64_t, ndim=3] simplices, float alpha=25):
    cdef np.ndarray[np.float64_t, ndim=1] pa
    cdef np.ndarray[np.float64_t, ndim=1] pb
    cdef np.ndarray[np.float64_t, ndim=1] pc
    cdef float a, b, c, s, area, circum_r
    cdef list triangles_to_delete
    cdef unsigned int index
    
    triangles_to_delete = []
    index = 0
    length = len(simplices)
    while index < length:
        pa = simplices[index, 0, :]
        pb = simplices[index, 1, :]
        pc = simplices[index, 2, :]
        # Lengths of sides of triangle
        a = sqrt((pa[0]-pb[0])**2 + (pa[1]-pb[1])**2)
        b = sqrt((pb[0]-pc[0])**2 + (pb[1]-pc[1])**2)
        c = sqrt((pc[0]-pa[0])**2 + (pc[1]-pa[1])**2)
        # Semiperimeter of triangle
        s = (a + b + c)/2.0
        # Area of triangle by Heronc's formula
        area = sqrt(s*(s-a)*(s-b)*(s-c))
        if area == 0.0:
            triangles_to_delete.append(index)
        else:
            circum_r = a*b*c/(4.0*area)
            # Here's the radius filter.
            if circum_r > alpha:
                triangles_to_delete.append(index)
        index += 1
    return triangles_to_delete

In [3]:
all_files = glob(TH_PATH + '*')
loaded = []
for filename in all_files:
    basename = os.path.splitext(os.path.basename(filename))[0]
    edited_path = TO_PATH + basename + '.pkl'
    rgb_image_path = glob(DATA_PATH + basename + '*')[0]
    edited = os.path.exists(edited_path)
    cls_id, cls = ch.get_class(basename)
    
    if edited:
        with gzip.GzipFile(edited_path, 'rb') as f:
            content = pickle.load(f)
            done = 'done' in content and content['done']
            bone_pixels = content['bone_pixels']
            markers = content['markers'] if 'markers' in content else {}
                
    else:
        done = False
        bone_pixels = io.imread(filename)
        markers = {}
    
    loaded.append({
        'filename': basename,
        'save_path': edited_path,
        'image': io.imread(rgb_image_path),
        'class_id': cls_id,
        'class_short': ch.get_class_short(cls_id),
        'class': cls,
        'done': done,
        'edited': edited,
        'bone_pixels': bone_pixels,
        'markers': markers
    })

In [4]:
dh.triangulation(loaded, do_triangulation)